# 1. Importing Necessary Libraries

In [156]:
import pandas as pd
import numpy as np
import seaborn as sns
import datetime as dt
import scipy.stats as st
import os

In [157]:
!pwd

/workspace/MEG_CHECK RELEASE/Notebook Files


# 2. Data-Cleaning our CSV

In [161]:
# Cleaning DataFrame from CSV
def load_dataframe(file_path, output_path="cleaned.csv"):
    use_cols = [
        'PAYEE', 'PROJECT', 'CHECK NO', 'AMOUNT', 'APV NO', 'CV NO',
        'PO NO', 'COMPANY', 'DATE OF INCLUSION IN THE LIST OF FOR RELEASE',
        'RELEASING VENUE'
    ]

    col_dtypes = {
                    'PAYEE': str,
                    'PROJECT': str,
                    'CHECK NO': str,
                    'APV NO': str,
                    'CV NO': str,
                    'PO NO': str,
                    'COMPANY': str,
                    'RELEASING VENUE': str,
                    'PAYEE_CODE': str
                }
    
    # Load depending on file type
    if file_path.endswith(".csv"):
        df = pd.read_csv(file_path, dtype=col_dtypes, skiprows=12, low_memory=True, usecols=use_cols)
    elif file_path.endswith((".xls", ".xlsx")):
        df = pd.read_excel(file_path, usecols=use_cols, skiprows=12)
        df = df.astype(col_dtypes)  # enforce dtypes manually
    else:
        raise ValueError("Unsupported file type. Use CSV or Excel.")

    # Convert date column
    df["DATE OF INCLUSION IN THE LIST OF FOR RELEASE"] = pd.to_datetime(
    df["DATE OF INCLUSION IN THE LIST OF FOR RELEASE"].str.strip(),
    format="%m/%d/%Y", errors="coerce")
    
    # Clean rows
    df_clean = df[df["DATE OF INCLUSION IN THE LIST OF FOR RELEASE"].notna()].copy()
    df_clean = df_clean[~(df_clean == df_clean.columns).all(axis=1)]

    df_clean["AMOUNT"] = (
                        df_clean["AMOUNT"]
                        .astype(str)                       # ensure string
                        .str.replace(",", "", regex=True)  # remove commas
                        .str.replace(r"[^\d.]", "", regex=True)  # strip non-numeric except .
                        .replace("", np.nan)               # turn empty strings into NaN
                        .astype(float)                     # convert to float
                    )
 
    # Split PAYEE into PAYEE and PAYEE_CODE
    df_clean["BILLING NO."] = df_clean["PAYEE"].str.extract(r"\((.*?)\)")
    df_clean["PAYEE"] = df_clean["PAYEE"].str.replace(r"\s*\(.*?\)", "", regex=True)

    # Checking if Column Types
    col_dtype_map = df_clean.dtypes
    print(col_dtype_map)
    print(df_clean.isna().sum())

    df_clean.to_csv(output_path, index=False)
    return df_clean
    
# Using function load_dataframe
df_clean = load_dataframe(r"/workspace/MEG_CHECK RELEASE/RAW_DATASETS/JANUARY 2026/List of checks for release January 30, 2026.csv", 
                          output_path=r"/workspace/MEG_CHECK RELEASE/CLEANED_DATASETS/Meg_CheckRelease_01-30-2026.csv")

PAYEE                                                   object
PROJECT                                                 object
CHECK NO                                                object
AMOUNT                                                 float64
APV NO                                                  object
CV NO                                                   object
PO NO                                                   object
COMPANY                                                 object
DATE OF INCLUSION IN THE LIST OF FOR RELEASE    datetime64[ns]
RELEASING VENUE                                         object
BILLING NO.                                             object
dtype: object
PAYEE                                             0
PROJECT                                           0
CHECK NO                                          0
AMOUNT                                            0
APV NO                                            0
CV NO                            

## 2.1 Cleaning Amount due to more than 1 "." in a row

In [153]:
use_cols = [
        'PAYEE', 'PROJECT', 'CHECK NO', 'AMOUNT', 'APV NO', 'CV NO',
        'PO NO', 'COMPANY', 'DATE OF INCLUSION IN THE LIST OF FOR RELEASE',
        'RELEASING VENUE'
    ]

df = pd.read_csv("/workspace/MEG_CHECK RELEASE/RAW_DATASETS/JANUARY 2026/List of checks for release January 30, 2026.csv", 
                 skiprows=14, low_memory=True, usecols=use_cols)

# cant = df.loc[df["AMOUNT"] == "1152.634.20"]
# print(cant)

print(df[df["AMOUNT"].str.count(r"\.") > 1])

                                         PAYEE         PROJECT     CHECK NO  \
344  MORETHANALL INDUSTRIAL TRADING INC. (B#2)  9 Central Park  CHECKWRITER   

            AMOUNT         APV NO          CV NO          PO NO  \
344  1,152.634.20   012026-00032   012026-00032   NWI-000000708   

                     COMPANY DATE OF INCLUSION IN THE LIST OF FOR RELEASE  \
344  NORTHWIN PROPERTIES INC                                   01/30/2026   

                                       RELEASING VENUE  
344  UBP MCKINLEY WEST – LAWTON AVENUE LOWER G/F, R...  


/tmp/ipykernel_28/1819231755.py:7: DtypeWarning: Columns (1,3,4,5,6,7,8,9,10,11) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/workspace/MEG_CHECK RELEASE/RAW_DATASETS/JANUARY 2026/List of checks for release January 30, 2026.csv",


## 2.2 Recursive Cleaning Before Loading

In [170]:
import os
import pandas as pd

raw_csv_path = "MEG_CHECK RELEASE/RAW_DATASETS"
cleaned_output_path = "MEG_CHECK RELEASE/CLEANED_DATASETS"
merged_output_file = "MEG_CHECK RELEASE/MERGED_DATA.csv"

def recursive_load_dataframe(raw_csv_path=raw_csv_path, cleaned_output_path=cleaned_output_path):
    """
    Recursively load all CSVs from raw_csv_path,
    save copies to cleaned_output_path.
    """
    if not os.path.exists(cleaned_output_path):
        os.makedirs(cleaned_output_path)
    
    for root, dirs, files in os.walk(raw_csv_path):
        for file in files:
            if file.lower().endswith(".csv") and "-checkpoint" not in file.lower():
                csv_file_path = os.path.join(root, file)
                print(f"Found CSV: {csv_file_path}")
                
                try:
                    df = pd.read_csv(csv_file_path)
                    print(f"Loaded {len(df)} rows from {file}")
                    
                    # Save copy to cleaned_output_path
                    output_file_path = os.path.join(cleaned_output_path, file)
                    df.to_csv(output_file_path, index=False)
                    print(f"Saved to {output_file_path}")
                    
                except Exception as e:
                    print(f"Failed to load {csv_file_path}: {e}")

def merge_all_csvs(cleaned_output_path=cleaned_output_path, merged_output_file=merged_output_file):
    """
    Merge only main CSV files (ignore checkpoint files).
    """
    all_dfs = []
    
    for root, dirs, files in os.walk(cleaned_output_path):
        for file in files:
            # Ignore checkpoint files
            if file.lower().endswith(".csv") and "-checkpoint" not in file.lower():
                
                csv_file_path = os.path.join(root, file)
                
                try:
                    df = pd.read_csv(csv_file_path)
                    all_dfs.append(df)
                    print(f"Added {file} with {len(df)} rows to merge")
                
                except Exception as e:
                    print(f"Failed to read {csv_file_path}: {e}")
    
    if all_dfs:
        merged_df = pd.concat(all_dfs, ignore_index=True)
        merged_df.to_csv(merged_output_file, index=False)
        print(f"Merged CSV saved to {merged_output_file} with {len(merged_df)} rows")
    else:
        print("No CSV files found to merge.")

# Step 1: Load CSVs recursively and save copies of each file
# recursive_load_dataframe(raw_csv_path, cleaned_output_path)

# Step 2: Merge all cleaned CSVs into one CSV
merge_all_csvs("/workspace/MEG_CHECK RELEASE/CLEANED_DATASETS/JANUARY 2026", "/workspace/MEG_CHECK RELEASE/CLEANED_DATASETS_JAN.csv")

Added Meg_CheckRelease_01-23-2026.csv with 534 rows to merge
Added Meg_CheckRelease_01-16-2026.csv with 691 rows to merge
Added Meg_CheckRelease_01-30-2026.csv with 417 rows to merge
Added Meg_CheckRelease_01-09-2026.csv with 591 rows to merge
Merged CSV saved to /workspace/MEG_CHECK RELEASE/CLEANED_DATASETS_JAN.csv with 2233 rows


# 3. Connecting to PostgreSQL

In [171]:
import psycopg2

In [173]:
import psycopg2

with psycopg2.connect(
    host="host.docker.internal",
    port=5432,
    database="MegaworldDB",
    user="admin",
    password="123456789",
    sslmode="prefer"
) as conn:
    # Use a context manager for the cursor
    with conn.cursor() as cur:
        csv_file_path = "/workspace/MEG_CHECK RELEASE/CLEANED_DATASETS_FEB_2026.csv"
        
        # Import CSV into table
        with open(csv_file_path, 'r') as f:
            cur.copy_expert(
                """
                COPY check_release.records(
                    "PAYEE",
                    "PROJECT",
                    "CHECK NO",
                    "AMOUNT",
                    "APV NO",
                    "CV NO",
                    "PO NO",
                    "COMPANY",
                    "DATE OF INCLUSION IN THE LIST OF FOR RELEASE",
                    "RELEASING VENUE",
                    "BILLING NO."
                )
                FROM STDIN WITH CSV HEADER
                """,
                f
            )
        
        # Commit is automatic when using 'with conn', but explicit commit is fine
        conn.commit()

print("CSV imported successfully!")

CSV imported successfully!


In [ ]:
with conn.cursor() as cur:
            cur.execute("""
                CREATE SCHEMA IF NOT EXISTS Check_Release;

                CREATE TABLE IF NOT EXISTS Check_Release.Records (
                    "PAYEE" TEXT,
                    "PROJECT" TEXT,
                    "CHECK NO" TEXT,
                    "AMOUNT" NUMERIC,
                    "APV NO" TEXT,
                    "CV NO" TEXT,
                    "PO NO" TEXT,
                    "COMPANY" TEXT,
                    "DATE OF INCLUSION IN THE LIST OF FOR RELEASE" DATE,
                    "RELEASING VENUE" TEXT,
                    "BILLING NO." TEXT
                );
            """)
            conn.commit()

# **WE CAN NOW PROCEED OUR ANALYSIS IN POSTGRESQL - http://localhost:5050/browser/**